In [2]:
#!pip install duckdb --proxy=http://proxyvip.national.agri:8080
#!pip install httpfs --proxy=http://proxyvip.national.agri:8080

In [1]:
set HTTP_PROXYS=https://proxyvip.national.agri:8080
set HTTP_PROXY=http://proxyvip.national.agri:8080

SyntaxError: invalid syntax (Temp/ipykernel_18184/4002611989.py, line 1)

In [1]:
import pandas as pd
import geopandas as gpd
import duckdb 

In [2]:
duckdb.sql('LOAD httpfs ;INSTALL spatial; LOAD spatial;')

In [3]:
url = "http://minio.lab.sspcloud.fr/h529p3/data/Filosofi2017_carreaux_200m_met.parquet"
url = "Filosofi2017_carreaux_200m_met.parquet"
#url = "http://minio.lab.sspcloud.fr/h529p3/data/Filosofi2017_carreaux_200m_partition/*/*.parquet'"
#url ="https://static.data.gouv.fr/resources/sirene-geolocalise-parquet/20240107-143656/sirene2024-geo.parquet"
#url ="https://static.data.gouv.fr/resources/recensement-de-la-population-fichiers-detail-individus-localises-au-canton-ou-ville-2020-1/20231023-122841/fd-indcvi-2020.parquet"

In [4]:
duckdb.sql(f'''
  CREATE OR REPLACE VIEW carreaux
  AS SELECT * FROM read_parquet("{url}") 
'''
)

Requetage des carreaux à 1km de notre dame

In [5]:
latitude = 48.853
longitude = 2.35
rayon = 1000

In [159]:
coordonnees_ctr_hubblo = duckdb.sql(
f''' 
select st_x(geo) as x, st_y(geo) as y from
(select ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154') as geo) ''').df()
x_hubblo = coordonnees_ctr_hubblo.x[0]
y_hubblo = coordonnees_ctr_hubblo.y[0]

In [174]:
duckdb.sql(
f'''

select sum(Ind*coeff) as Ind,
       sum(Men_1ind*coeff) as Men_1ind, 
       sum(Men_5ind*coeff) as Men_5ind, 
       sum(Men_prop*coeff) as Men_prop, 
       sum(Men_fmp*coeff) as Men_fmp,
       sum(Ind_snv*coeff) as Ind_snv, 
       sum(Men_surf*coeff) as Men_surf, 
       sum(Men_coll*coeff) as Men_coll, 
       sum(Men_mais*coeff) as Men_mais, 
       sum(Log_av45*coeff) as Log_av45, 
       sum(Log_45_70*coeff) as Log_45_70,
       sum(Log_70_90*coeff) as Log_70_90, 
       sum(Log_ap90*coeff) as Log_ap90, 
       sum(Log_inc*coeff) as Log_inc, 
       sum(Log_soc*coeff) as Log_soc, 
       sum(Ind_0_3*coeff) as Ind_0_3, 
       sum(Ind_4_5*coeff) as Ind_4_5,
       sum(Ind_6_10*coeff) as Ind_6_10, 
       sum(Ind_11_17*coeff) as Ind_11_17, 
       sum(Ind_18_24*coeff) as Ind_18_24, 
       sum(Ind_25_39*coeff) as Ind_25_39, 
       sum(Ind_40_54*coeff) as Ind_40_54,
       sum(Ind_55_64*coeff) as Ind_55_64, 
       sum(Ind_65_79*coeff) as Ind_65_79, 
       sum(Ind_80p*coeff) as Ind_80p, 
       sum(Ind_inc*coeff) as Ind_inc,  
       sum(Men_pauv*coeff) as Men_pauv,
       sum(Men*coeff) as Men 
       
from 
(
select *, st_area(st_intersection(ST_GEOMFROMWKB(geometry), 
         (SELECT ST_BUFFER(ST_POINT({x_hubblo},{y_hubblo}),{rayon}))))/st_area(ST_GEOMFROMWKB(geometry)) as coeff  
         from  
(select * from carreaux
where carreaux.x > ({x_hubblo}-{rayon}-200) and
      carreaux.x < ({x_hubblo}+{rayon}+200) and
      carreaux.y > ({y_hubblo}-{rayon}-200) and
      carreaux.y < ({y_hubblo}+{rayon}+200)) as temp
where st_intersects(ST_GEOMFROMWKB(temp.geometry),(SELECT ST_BUFFER(geo,{rayon}) from ctr_hubblo)))
'''
)

┌───────────────────┬────────────────────┬───────────────────┬───┬────────────────────┬────────────────────┐
│        Ind        │      Men_1ind      │     Men_5ind      │ … │      Men_pauv      │        Men         │
│      double       │       double       │      double       │   │       double       │       double       │
├───────────────────┼────────────────────┼───────────────────┼───┼────────────────────┼────────────────────┤
│ 49333.70344815229 │ 14808.528829264807 │ 937.1438321337375 │ … │ 4077.6481360048256 │ 27396.886667738083 │
├───────────────────┴────────────────────┴───────────────────┴───┴────────────────────┴────────────────────┤
│ 1 rows                                                                              28 columns (5 shown) │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [119]:
duckdb.sql('''from carreaux 
            select * limit 10; ''').df().columns

Index(['Idcar_200m', 'I_est_200', 'Idcar_1km', 'I_est_1km', 'Idcar_nat',
       'Groupe', 'Ind', 'Men_1ind', 'Men_5ind', 'Men_prop', 'Men_fmp',
       'Ind_snv', 'Men_surf', 'Men_coll', 'Men_mais', 'Log_av45', 'Log_45_70',
       'Log_70_90', 'Log_ap90', 'Log_inc', 'Log_soc', 'Ind_0_3', 'Ind_4_5',
       'Ind_6_10', 'Ind_11_17', 'Ind_18_24', 'Ind_25_39', 'Ind_40_54',
       'Ind_55_64', 'Ind_65_79', 'Ind_80p', 'Ind_inc', 'Men_pauv', 'Men',
       'lcog_geo', 'geometry', 'x', 'y', '__index_level_0__',
       '__index_level_1__', '__index_level_2__'],
      dtype='object')

In [35]:
duckdb.sql(
f'''
select * from
(select  ST_Transform(ST_Point({48.853} , {2.35}),'EPSG:4326', 'EPSG:2154') as geo), carreaux 
where (abs(carreaux.x - ST_X(geo))<1200 and (abs(carreaux.y - ST_Y(geo))<1200))

'''
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬──────────────────────┬───────────┬───┬───────────────────┬────────────────────┐
│         geo          │      Idcar_200m      │ I_est_200 │ … │ __index_level_1__ │ __index_level_2__  │
│       geometry       │       varchar        │   int64   │   │      double       │       double       │
├──────────────────────┼──────────────────────┼───────────┼───┼───────────────────┼────────────────────┤
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │  651325.286367202 │  6862743.843861913 │
│ POINT (652304.3107…  │ CRS3035RES200mN288…  │         0 │ … │ 651960.2608834291 │  6862403.233705901 │
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │ 651524.4928184762 │  6862762.950441208 │
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │ 651723.6992050873 │  6862782.056535732 │
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │ 651922.9055270783 │  6862801.162145468 │
│ POINT (652304.3107…  │ CRS3035RES200mN288…  │        

In [44]:
duckdb.sql(
f'''
select * from
(select  ST_Transform(ST_Point({48.853} , {2.35}),'EPSG:4326', 'EPSG:2154') as geo), carreaux 
where (carreaux.x > (ST_X(geo) - 1200)) and
      (carreaux.x < (ST_X(geo) + 1200)) and
      (carreaux.y > (ST_Y(geo) - 1200)) and
      (carreaux.y < (ST_Y(geo) + 1200)) ;


'''
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬──────────────────────┬───────────┬───┬───────────────────┬────────────────────┐
│         geo          │      Idcar_200m      │ I_est_200 │ … │ __index_level_1__ │ __index_level_2__  │
│       geometry       │       varchar        │   int64   │   │      double       │       double       │
├──────────────────────┼──────────────────────┼───────────┼───┼───────────────────┼────────────────────┤
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │  651325.286367202 │  6862743.843861913 │
│ POINT (652304.3107…  │ CRS3035RES200mN288…  │         0 │ … │ 651960.2608834291 │  6862403.233705901 │
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │ 651524.4928184762 │  6862762.950441208 │
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │ 651723.6992050873 │  6862782.056535732 │
│ POINT (652304.3107…  │ CRS3035RES200mN289…  │         0 │ … │ 651922.9055270783 │  6862801.162145468 │
│ POINT (652304.3107…  │ CRS3035RES200mN288…  │        

In [59]:
a=duckdb.sql(
f'''
create or replace table ctr_hubblo as 
(select ST_Transform(ST_Point({48.853} , {2.35}),'EPSG:4326', 'EPSG:2154') as geo );

create or replace table hubblo_ag as 
select sum(Ind) from carreaux
where carreaux.x >(select st_x(geo)-1200 from ctr_hubblo2) and
      carreaux.x <(select st_x(geo)+1200 from ctr_hubblo2) and
      carreaux.y >(select st_y(geo)-1200 from ctr_hubblo2) and
      carreaux.y <(select st_y(geo)+1200 from ctr_hubblo2); 

'''
)

In [79]:
duckdb.sql(
f'''
create or replace table ctr_hubblo as 
(select ST_Transform(ST_Point({48.853} , {2.35}),'EPSG:4326', 'EPSG:2154') as geo );

select sum(Ind) from  
(select * from carreaux
where carreaux.x >(select st_x(geo)-1200 from ctr_hubblo2) and
      carreaux.x <(select st_x(geo)+1200 from ctr_hubblo2) and
      carreaux.y >(select st_y(geo)-1200 from ctr_hubblo2) and
      carreaux.y <(select st_y(geo)+1200 from ctr_hubblo2)) as temp
where st_intersects(ST_GEOMFROMWKB(temp.geometry),(SELECT ST_BUFFER(geo,1000) from ctr_hubblo))

'''
)

┌──────────┐
│ sum(Ind) │
│  double  │
├──────────┤
│  64826.0 │
└──────────┘

In [ ]:
duckdb.sql(
f'''
create or replace table ctr_hubblo as 
(select ST_Transform(ST_Point({48.853} , {2.35}),'EPSG:4326', 'EPSG:2154') as geo );

select sum(Ind) from  
(select * from carreaux
where carreaux.x >(select st_x(geo)-1200 from ctr_hubblo2) and
      carreaux.x <(select st_x(geo)+1200 from ctr_hubblo2) and
      carreaux.y >(select st_y(geo)-1200 from ctr_hubblo2) and
      carreaux.y <(select st_y(geo)+1200 from ctr_hubblo2)) as temp
where st_intersects(ST_GEOMFROMWKB(temp.geometry),(SELECT ST_BUFFER(geo,1000) from ctr_hubblo))

'''
)

In [136]:
duckdb.sql(
f'''
create or replace table ctr_hubblo as 
(select ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154') as geo );

select sum(Ind*coeff) as Ind,
       sum(Men*coeff) as Men  
from 
(
select *, st_area(st_intersection(ST_GEOMFROMWKB(geometry), (SELECT ST_BUFFER(geo,1000) from ctr_hubblo)))/st_area(ST_GEOMFROMWKB(geometry)) as coeff  from  
(select * from carreaux
where carreaux.__index_level_1__ >(select st_x(geo)-{rayon}-200 from ctr_hubblo2) and
      carreaux.__index_level_1__ <(select st_x(geo)+{rayon}+200 from ctr_hubblo2) and
      carreaux.__index_level_2__ >(select st_y(geo)-{rayon}-200 from ctr_hubblo2) and
      carreaux.__index_level_2__ <(select st_y(geo)+{rayon}+200 from ctr_hubblo2)) as temp
where st_intersects(ST_GEOMFROMWKB(temp.geometry),(SELECT ST_BUFFER(geo,{rayon}) from ctr_hubblo)))
'''
)

┌───────────────────┬────────────────────┐
│        Ind        │        Men         │
│      double       │       double       │
├───────────────────┼────────────────────┤
│ 49333.70344815229 │ 27396.886667738083 │
└───────────────────┴────────────────────┘

In [153]:
coordonnees_ctr_hubblo = duckdb.sql(
f''' 
select st_x(geo) as x, st_y(geo) as y from
(select ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154') as geo) ''').df()
coordonnees_ctr_hubblo_x = coordonnees_ctr_hubblo.x[0]
coordonnees_ctr_hubblo_y = coordonnees_ctr_hubblo.y[0]

In [138]:
a=duckdb.sql(
f'''
create or replace table ctr_hubblo as 
(select ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154') as geo );

select sum(Ind*coeff) as Ind,
       sum(Men_1ind*coeff) as Men_1ind, 
       sum(Men_5ind*coeff) as Men_5ind, 
       sum(Men_prop*coeff) as Men_prop, 
       sum(Men_fmp*coeff) as Men_fmp,
       sum(Ind_snv*coeff) as Ind_snv, 
       sum(Men_surf*coeff) as Men_surf, 
       sum(Men_coll*coeff) as Men_coll, 
       sum(Men_mais*coeff) as Men_mais, 
       sum(Log_av45*coeff) as Log_av45, 
       sum(Log_45_70*coeff) as Log_45_70,
       sum(Log_70_90*coeff) as Log_70_90, 
       sum(Log_ap90*coeff) as Log_ap90, 
       sum(Log_inc*coeff) as Log_inc, 
       sum(Log_soc*coeff) as Log_soc, 
       sum(Ind_0_3*coeff) as Ind_0_3, 
       sum(Ind_4_5*coeff) as Ind_4_5,
       sum(Ind_6_10*coeff) as Ind_6_10, 
       sum(Ind_11_17*coeff) as Ind_11_17, 
       sum(Ind_18_24*coeff) as Ind_18_24, 
       sum(Ind_25_39*coeff) as Ind_25_39, 
       sum(Ind_40_54*coeff) as Ind_40_54,
       sum(Ind_55_64*coeff) as Ind_55_64, 
       sum(Ind_65_79*coeff) as Ind_65_79, 
       sum(Ind_80p*coeff) as Ind_80p, 
       sum(Ind_inc*coeff) as Ind_inc,  
       sum(Men_pauv*coeff) as Men_pauv,
       sum(Men*coeff) as Men 
       
from 
(
select *, st_area(st_intersection(ST_GEOMFROMWKB(geometry), (SELECT ST_BUFFER(geo,1000) from ctr_hubblo)))/st_area(ST_GEOMFROMWKB(geometry)) as coeff  from  
(select * from carreaux
where carreaux.x >(select st_x(geo)-{rayon}-200 from ctr_hubblo2) and
      carreaux.x <(select st_x(geo)+{rayon}+200 from ctr_hubblo2) and
      carreaux.y >(select st_y(geo)-{rayon}-200 from ctr_hubblo2) and
      carreaux.y <(select st_y(geo)+{rayon}+200 from ctr_hubblo2)) as temp
where st_intersects(ST_GEOMFROMWKB(temp.geometry),(SELECT ST_BUFFER(geo,{rayon}) from ctr_hubblo)))
'''
)

In [71]:
%timeit 10
duckdb.sql(
f'''select sum(Ind) from carreaux, (SELECT ST_BUFFER(ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154'),{rayon}) AS hubblo) '''\
"where st_intersects(ST_GEOMFROMWKB(carreaux.geometry), hubblo)"

)

14.7 ns ± 2.39 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


┌──────────┐
│ sum(Ind) │
│  double  │
├──────────┤
│  64826.0 │
└──────────┘

In [13]:
duckdb.sql(f'''select ST_X(geo), ST_Y(geo) from  ctr_hubblo2''')

┌───────────────────┬───────────────────┐
│     st_x(geo)     │     st_y(geo)     │
│      double       │      double       │
├───────────────────┼───────────────────┤
│ 652304.3107785692 │ 6861636.298107567 │
└───────────────────┴───────────────────┘

In [29]:
%timeit 3
a=duckdb.sql(
f'''

select * from carreaux
where carreaux.x >(652304.3107785692 - 1200) and
      carreaux.x <(652304.3107785692 + 1200) and
      carreaux.y >(6861636.298107567-1200) and
      carreaux.y <(6861636.298107567+1200) 



'''
)

14.4 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [38]:
a=duckdb.sql(
f'''

select sum(Ind) from carreaux
where carreaux.x >(652304.3107785692 - 1200) and
      carreaux.x <(652304.3107785692 + 1200) and
      carreaux.y >(6861636.298107567-1200) and
      carreaux.y <(6861636.298107567+1200); 



'''
)

In [39]:
a.df()

,sum(Ind)
0,87102.0


In [10]:
duckdb.sql(
f'''select * from carreaux, (SELECT ST_BUFFER(ST_Transform(ST_Point({48.853} , {2.35}),'EPSG:4326', 'EPSG:2154'),{1000}) AS hubblo) '''\
"where st_intersects(ST_GEOMFROMWKB(carreaux.geometry), hubblo)"
)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬───────────┬──────────────────────┬───┬────────────────────┬──────────────────────┐
│      Idcar_200m      │ I_est_200 │      Idcar_1km       │ … │ __index_level_2__  │        hubblo        │
│       varchar        │   int64   │       varchar        │   │       double       │       geometry       │
├──────────────────────┼───────────┼──────────────────────┼───┼────────────────────┼──────────────────────┤
│ CRS3035RES200mN288…  │         0 │ CRS3035RES1000mN28…  │ … │  6862403.233705901 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ CRS3035RES1000mN28…  │ … │  6862384.128485699 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ CRS3035RES1000mN28…  │ … │  6862365.022780819 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN289…  │         0 │ CRS3035RES1000mN28…  │ … │  6862602.197852682 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN289…  │         0 │ CRS3035RES1000mN28…  │ … │  6862621.302782882 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN289…  │    

In [9]:
duckdb.sql(
f'''select * from carreaux, (SELECT ST_BUFFER(ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154'),{rayon}) AS hubblo) '''\
"where st_intersects(ST_GEOMFROMWKB(carreaux.geometry), hubblo)"
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬───────────┬──────────────────────┬───┬────────────────────┬──────────────────────┐
│      Idcar_200m      │ I_est_200 │      Idcar_1km       │ … │ __index_level_2__  │        hubblo        │
│       varchar        │   int64   │       varchar        │   │       double       │       geometry       │
├──────────────────────┼───────────┼──────────────────────┼───┼────────────────────┼──────────────────────┤
│ CRS3035RES200mN288…  │         0 │ CRS3035RES1000mN28…  │ … │  6862403.233705901 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ CRS3035RES1000mN28…  │ … │  6862384.128485699 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ CRS3035RES1000mN28…  │ … │  6862365.022780819 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN289…  │         0 │ CRS3035RES1000mN28…  │ … │  6862602.197852682 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN289…  │         0 │ CRS3035RES1000mN28…  │ … │  6862621.302782882 │ POLYGON ((653304.3…  │
│ CRS3035RES200mN289…  │    

In [30]:
duckdb.sql(
f'''select substr(lcog_geo,1,2) as 'dep', sum(Ind) as 'pop' from carreaux group by all order by dep;'''
)

┌─────────┬───────────┐
│   dep   │    pop    │
│ varchar │  double   │
├─────────┼───────────┤
│ 01      │  624195.5 │
│ 02      │  520062.0 │
│ 03      │  330110.5 │
│ 04      │  157409.5 │
│ 05      │  135605.5 │
│ 06      │ 1110195.5 │
│ 07      │  315211.0 │
│ 08      │  265642.0 │
│ 09      │  145288.5 │
│ 10      │  293580.5 │
│ ·       │      ·    │
│ ·       │      ·    │
│ ·       │      ·    │
│ 86      │  409813.0 │
│ 87      │  350617.0 │
│ 88      │  359006.0 │
│ 89      │  329131.0 │
│ 90      │  134097.0 │
│ 91      │ 1278632.0 │
│ 92      │ 1576971.5 │
│ 93      │ 1583813.0 │
│ 94      │ 1342502.0 │
│ 95      │ 1238555.0 │
├─────────┴───────────┤
│ 96 rows (20 shown)  │
└─────────────────────┘

In [14]:
lgeo_com

NameError: name 'lgeo_com' is not defined